pytorch関連のインストール

In [1]:
!pip3 install torch torchvision torchtext

    100% |████████████████████████████████| 484.0MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b8e0000 @  0x7f87a431e1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 11.2MB/s 
    100% |████████████████████████████████| 51kB 12.4MB/s 
    100% |████████████████████████████████| 2.0MB 9.3MB/s 
    100% |████████████████████████████████| 51kB 12.4MB/s 
  Running setup.py bdist_wheel for torchtext ... - \

 done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Preparing to unpack .../04-gnupg_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg (2.1.15-1ubuntu8.1) over (2.1.15-1ubuntu8) ...
Preparing to unpack .../05-gnupg-agent_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg-agent (2.1.15-1ubuntu8

Setting up gir1.2-glib-2.0:amd64 (1.54.1-1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up gnupg-agent (2.1.15-1ubuntu8.1) ...
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod 

In [3]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
path = "drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch"
os.chdir(path)

In [6]:
!pwd

/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch


In [0]:
import os
import sys
import time
import glob

import torch
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets

from model import SNLIClassifier
from util import get_args, makedirs

In [8]:
sys.argv = [sys.argv[0], "--epochs", "10", '--d_embed', '100', "--word_vectors", "glove.6B.100d", '--vector_cache', '../input/vector_cache', '--train_embed']
args = get_args()
args
# !python train.py --epochs 5 --d_embed 300 --gpu -1 --word_vectors glove.6B.300d

Namespace(batch_size=128, birnn=True, d_embed=100, d_hidden=300, d_proj=300, dev_every=1000, dp_ratio=0.2, epochs=10, fix_emb=False, gpu=0, log_every=50, lower=True, lr=0.001, n_layers=1, projection=True, resume_snapshot='', save_every=1000, save_path='results', vector_cache='../input/vector_cache', word_vectors='glove.6B.100d')

In [0]:
torch.cuda.set_device(args.gpu)

In [0]:
inputs = data.Field(lower=args.lower)
answers = data.Field(sequential=False)
train, dev, test = datasets.SNLI.splits(inputs, answers, root="../input")

In [14]:
inputs.build_vocab(train, dev, test)

inputs.vocab.load_vectors(args.word_vectors, cache=args.vector_cache)
# if args.word_vectors:
#     if os.path.isfile(args.vector_cache):
#         inputs.vocab.vectors = torch.load(args.vector_cache)
#     else:
#         inputs.vocab.load_vectors(args.word_vectors)
#         makedirs(os.path.dirname(args.vector_cache))
#         torch.save(inputs.vocab.vectors, args.vector_cache)

answers.build_vocab(train)

100%|██████████| 400000/400000 [00:19<00:00, 20698.04it/s]


In [0]:
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=args.batch_size, device=args.gpu)

In [0]:
config = args
config.n_embed = len(inputs.vocab)
config.d_out = len(answers.vocab)
config.n_cells = config.n_layers

# double the number of cells for bidirectional networks
if config.birnn:
    config.n_cells *= 2

if args.resume_snapshot:
    model = torch.load(args.resume_snapshot, map_location=lambda storage, locatoin: storage.cuda(args.gpu))
else:
    model = SNLIClassifier(config)
    if args.word_vectors:
        model.embed.weight.data.copy_(inputs.vocab.vectors)
        model.cuda(args.gpu)

criterion = nn.CrossEntropyLoss()
opt = O.Adam(model.parameters(), lr=args.lr)

In [18]:
iterations = 0
start = time.time()
best_dev_acc = -1
train_iter.repeat = False
header = '  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy'
dev_log_template = ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{:8.6f},{:12.4f},{:12.4f}'.split(','))
log_template =     ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{},{:12.4f},{}'.split(','))
makedirs(args.save_path)
print(header)

for epoch in range(args.epochs):
    train_iter.init_epoch()
    n_correct, n_total = 0, 0
    for batch_idx, batch in enumerate(train_iter):

        # switch model to training mode, clear gradient accumulators
        model.train(); opt.zero_grad()

        iterations += 1

        # forward pass
        answer = model(batch)

        # calculate accuracy of predictions in the current batch
        n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
        n_total += batch.batch_size
        train_acc = 100. * n_correct/n_total

        # calculate loss of the network output with respect to training labels
        loss = criterion(answer, batch.label)

        # backpropagate and update optimizer learning rate
        loss.backward(); opt.step()

        # checkpoint model periodically
        if iterations % args.save_every == 0:
            snapshot_prefix = os.path.join(args.save_path, 'snapshot')
            snapshot_path = snapshot_prefix + '_acc_{:.4f}_loss_{:.6f}_iter_{}_model.pt'.format(train_acc, loss.item(), iterations)
            torch.save(model, snapshot_path)
            for f in glob.glob(snapshot_prefix + '*'):
                if f != snapshot_path:
                    os.remove(f)

        # evaluate performance on validation set periodically
        if iterations % args.dev_every == 0:

            # switch model to evaluation mode
            model.eval(); dev_iter.init_epoch()

            # calculate accuracy on validation set
            n_dev_correct, dev_loss = 0, 0
            with torch.no_grad():
                for dev_batch_idx, dev_batch in enumerate(dev_iter):
                     answer = model(dev_batch)
                     n_dev_correct += (torch.max(answer, 1)[1].view(dev_batch.label.size()) == dev_batch.label).sum().item()
                     dev_loss = criterion(answer, dev_batch.label)
            dev_acc = 100. * n_dev_correct / len(dev)

            print(dev_log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), dev_loss.item(), train_acc, dev_acc))

            # update best valiation set accuracy
            if dev_acc > best_dev_acc:

                # found a model with better validation set accuracy

                best_dev_acc = dev_acc
                snapshot_prefix = os.path.join(args.save_path, 'best_snapshot')
                snapshot_path = snapshot_prefix + '_devacc_{}_devloss_{}__iter_{}_model.pt'.format(dev_acc, dev_loss.item(), iterations)

                # save model, delete previous 'best_snapshot' files
                torch.save(model, snapshot_path)
                for f in glob.glob(snapshot_prefix + '*'):
                    if f != snapshot_path:
                        os.remove(f)

        elif iterations % args.log_every == 0:

            # print progress message
            print(log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), ' '*8, n_correct/n_total*100, ' '*12))




  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy
     6     0        50    50/4292        1% 1.106509               33.5938             
    10     0       100   100/4292        2% 1.080925               34.3047             
    13     0       150   150/4292        3% 0.977589               37.9583             
    16     0       200   200/4292        5% 1.059282               40.6953             
    20     0       250   250/4292        6% 0.997258               43.0219             
    23     0       300   300/4292        7% 0.880332               45.2083             
    26     0       350   350/4292        8% 0.857521               46.9933             
    30     0       400   400/4292        9% 0.998504               48.5215             
    33     0       450   450/4292       10% 0.899105               49.9167             
    37     0       500   500/4292       12% 0.850374               51.0141             
    40     0       550   550/42

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


    85     0      1000  1000/4292       23% 0.826093 0.919542      57.2516      66.0841
    98     0      1050  1050/4292       24% 0.714401               57.6012             
   101     0      1100  1100/4292       26% 0.774260               58.0021             
   105     0      1150  1150/4292       27% 0.769026               58.3356             
   108     0      1200  1200/4292       28% 0.759885               58.6823             
   111     0      1250  1250/4292       29% 0.870742               58.9988             
   115     0      1300  1300/4292       30% 0.752269               59.2776             
   118     0      1350  1350/4292       31% 0.807818               59.5579             
   121     0      1400  1400/4292       33% 0.662914               59.8214             
   125     0      1450  1450/4292       34% 0.731366               60.0932             
   128     0      1500  1500/4292       35% 0.749157               60.3458             
   131     0      1550  1550/429

   290     0      3350  3350/4292       78% 0.622584               65.4056             
   294     0      3400  3400/4292       79% 0.767217               65.4766             
   297     0      3450  3450/4292       80% 0.686264               65.5514             
   300     0      3500  3500/4292       82% 0.636470               65.6379             
   304     0      3550  3550/4292       83% 0.793130               65.7121             
   307     0      3600  3600/4292       84% 0.720002               65.7886             
   311     0      3650  3650/4292       85% 0.667529               65.8686             
   314     0      3700  3700/4292       86% 0.768202               65.9548             
   318     0      3750  3750/4292       87% 0.669939               66.0283             
   321     0      3800  3800/4292       89% 0.602688               66.1059             
   324     0      3850  3850/4292       90% 0.667558               66.1794             
   328     0      3900  3900/429

   492     1      5700  1408/4292       33% 0.548881               75.1970             
   495     1      5750  1458/4292       34% 0.615353               75.2031             
   498     1      5800  1508/4292       35% 0.653242               75.2129             
   502     1      5850  1558/4292       36% 0.625129               75.1966             
   505     1      5900  1608/4292       37% 0.575291               75.2002             
   509     1      5950  1658/4292       39% 0.660639               75.2111             
   526     1      6000  1708/4292       40% 0.666367 0.757223      75.1880      74.4564
   537     1      6050  1758/4292       41% 0.737975               75.1649             
   541     1      6100  1808/4292       42% 0.551321               75.1949             
   544     1      6150  1858/4292       43% 0.653301               75.2056             
   547     1      6200  1908/4292       44% 0.556510               75.2301             
   551     1      6250  1958/429

...

  2643     7     31550  1506/4292       35% 0.304459               88.5520             
  2647     7     31600  1556/4292       36% 0.255664               88.5092             
  2650     7     31650  1606/4292       37% 0.317817               88.5070             
  2654     7     31700  1656/4292       39% 0.352998               88.4940             
  2657     7     31750  1706/4292       40% 0.278843               88.4823             
  2660     7     31800  1756/4292       41% 0.292356               88.4734             
  2664     7     31850  1806/4292       42% 0.191707               88.4469             
  2667     7     31900  1856/4292       43% 0.338184               88.4391             
  2671     7     31950  1906/4292       44% 0.339846               88.4120             
  2684     7     32000  1956/4292       46% 0.358787 0.760765      88.3959      77.3522
  2688     7     32050  2006/4292       47% 0.348919               88.3876             
  2691     7     32100  2056/429

  2823     7     33900  3856/4292       90% 0.454221               87.8558             
  2827     7     33950  3906/4292       91% 0.408967               87.8412             
  2840     7     34000  3956/4292       92% 0.358193 0.692457      87.8316      77.1286
  2844     7     34050  4006/4292       93% 0.420624               87.8243             
  2847     7     34100  4056/4292       95% 0.324359               87.8169             
  2850     7     34150  4106/4292       96% 0.411568               87.8073             
  2854     7     34200  4156/4292       97% 0.402678               87.7919             
  2857     7     34250  4206/4292       98% 0.341852               87.7805             
  2860     7     34300  4256/4292       99% 0.429248               87.7715             
  2866     8     34350    14/4292        0% 0.271839               90.5134             
  2869     8     34400    64/4292        1% 0.151638               90.7227             
  2873     8     34450   114/429

  3017     8     36250  1914/4292       45% 0.339449               89.5780             
  3020     8     36300  1964/4292       46% 0.315531               89.5438             
  3024     8     36350  2014/4292       47% 0.285731               89.5202             
  3027     8     36400  2064/4292       48% 0.260659               89.4967             
  3030     8     36450  2114/4292       49% 0.258930               89.4831             
  3034     8     36500  2164/4292       50% 0.188557               89.4546             
  3037     8     36550  2214/4292       52% 0.397172               89.4450             
  3041     8     36600  2264/4292       53% 0.283206               89.4283             
  3044     8     36650  2314/4292       54% 0.323131               89.4190             
  3047     8     36700  2364/4292       55% 0.265278               89.4171             
  3051     8     36750  2414/4292       56% 0.314741               89.3923             
  3054     8     36800  2464/429

  3197     8     38600  4264/4292       99% 0.364528               88.9560             
  3202     9     38650    22/4292        1% 0.347088               91.9744             
  3206     9     38700    72/4292        2% 0.416915               91.6558             
  3209     9     38750   122/4292        3% 0.252863               91.5856             
  3213     9     38800   172/4292        4% 0.234948               91.7605             
  3216     9     38850   222/4292        5% 0.241266               91.5928             
  3220     9     38900   272/4292        6% 0.201489               91.4694             
  3223     9     38950   322/4292        8% 0.285646               91.4912             
  3236     9     39000   372/4292        9% 0.186055 0.929233      91.4588      77.0677
  3239     9     39050   422/4292       10% 0.241195               91.3526             
  3243     9     39100   472/4292       11% 0.203838               91.2689             
  3246     9     39150   522/429

  3379     9     40950  2322/4292       54% 0.386803               90.5002             
  3392     9     41000  2372/4292       55% 0.230410 0.942854      90.4916      76.4987
  3396     9     41050  2422/4292       56% 0.196738               90.4795             
  3399     9     41100  2472/4292       58% 0.332516               90.4799             
  3402     9     41150  2522/4292       59% 0.291483               90.4608             
  3406     9     41200  2572/4292       60% 0.172602               90.4549             
  3409     9     41250  2622/4292       61% 0.196437               90.4418             
  3413     9     41300  2672/4292       62% 0.261214               90.4215             
  3416     9     41350  2722/4292       63% 0.248003               90.4046             
  3419     9     41400  2772/4292       65% 0.392714               90.3950             
  3423     9     41450  2822/4292       66% 0.230152               90.3792             
  3426     9     41500  2872/429

google driveとの同期プロセスも完全にkillする

In [0]:
# !kill -9 -1